In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
sample_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

train, test 결측치, 전처리 분리는
piplining 쓰면 new data 는 무시가능...


In [ ]:
train.info()

In [ ]:
train.head(3)

In [ ]:
na_percent = (train.isna().sum()/len(train) * 100)
cols = train.columns

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (20,10)

pd.Series(na_percent, index = cols).plot(kind = 'bar')

In [ ]:
data = pd.concat([train,test], axis =0).copy()

In [ ]:
na_percent = (data.isna().sum()/len(data) * 100)
cols = data.columns
data = data.drop(['Id'], axis =1)
many_na_col = cols[na_percent >30]
data = data.drop(many_na_col, axis =1)

In [ ]:
na_percent = (data.isna().sum()/len(data) * 100)
cols = data.columns

plt.rcParams['figure.figsize'] = (20,10)

pd.Series(na_percent, index = cols).plot(kind = 'bar')

In [ ]:
cols

In [ ]:
na_cols = [(col) for col in cols[na_percent >0]]
data[na_cols].info()

In [ ]:
nadata = data[na_cols].copy()
na_cols = nadata.columns
o_col = na_cols[nadata.dtypes == 'object']
num_col = na_cols[nadata.dtypes != 'object']


nadata.boxplot('LotFrontage')

In [ ]:
data[["LotFrontage", "MasVnrArea", "GarageYrBlt"]].describe()

In [ ]:
for col in num_col :
       data[col].fillna(data[col].median(), inplace= True)

In [ ]:
data.fillna('ffill', inplace = True)
data.fillna('bfill', inplace = True)

In [ ]:
data[na_cols].info()

In [ ]:
data.isna().sum().sum()

In [ ]:
cols = data.columns

o_col = cols[data.dtypes == 'object']
num_col = cols[data.dtypes != 'object']


In [ ]:
num_col

In [ ]:
o_col

In [ ]:
dummy_data = data.copy()
for col in o_col:
    dummy_data = pd.get_dummies(dummy_data,columns = [col])

In [ ]:
dummy_data.info()

In [ ]:
dummy_data

In [ ]:
cleaned_train = dummy_data.iloc[:len(train)].copy()
cleaned_test = dummy_data.iloc[len(train):].copy()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
print(cleaned_train.shape)
print(cleaned_test.shape)

분석

In [ ]:
cleaned_train.corr()

In [ ]:
cleaned_train

In [ ]:
train_x = cleaned_train.copy()
train_y = train['SalePrice'].copy()
test_x = cleaned_test.copy()

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
train_x

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import ShuffleSplit, cross_validate

lasso = Lasso()

lasso = Lasso(alpha= 1)
cv = ShuffleSplit(n_splits= 50, test_size=0.2)

scores = cross_validate(lasso,
                           X = train_x,
                           y = train_y,
                           cv = cv,
                         return_train_score= True                       
                         
                          )

pd.DataFrame(scores)[['test_score','train_score']].mean(0)

from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate, ShuffleSplit

alphas = [100000, 10000, 1000,100, 10, 1, 0.1, 0.01]

train_scores = []
test_scores = []

for alpha in alphas:
    lasso = Lasso( alpha = alpha)
    cv = ShuffleSplit(n_splits= 50, test_size=0.2, random_state= 10)

    scores = cross_validate(lasso,
                             X = train_x,
                             y = train_y,
                             cv = cv,
                             return_train_score= True   
                              )
    test_mean_score, train_mean_score = pd.DataFrame(scores)[['test_score','train_score']].mean()
    test_scores.append(test_mean_score)
    train_scores.append(train_mean_score)
    

import matplotlib.pyplot as plt

plt.plot(range(len(alphas)), train_scores)
plt.plot(range(len(alphas)), test_scores)


plt.legend(['train','test'])
plt.show()

pd.DataFrame(
    {'train_scores' : train_scores,
     'test_scores' : test_scores,
     'alphas' : alphas
    }
)

In [ ]:
alpha = 1000

In [ ]:
lasso =Lasso(alpha = alpha)
lasso.fit(train_x, train_y)
predicted = lasso.predict(test_x)


In [ ]:
pd.Series(lasso.coef_, index = train_x.columns).plot(kind = 'bar')

In [ ]:
sample_submission['SalePrice'] = predicted

In [ ]:
sample_submission.to_csv('./lasso_submission.csv', index = False)

In [ ]:
sample_submission

random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit


ns = [100, 200, 300 ]
max_depths = [5, 10, 20, 30 ]
test_scores = []
train_scores = []

for n in ns:
    rf = RandomForestRegressor(n_estimators= n, random_state = 10, max_depth = None)
    cv = ShuffleSplit(n_splits= 50, test_size=0.2, random_state= 10)

    scores = cross_validate(rf,
                             X = train_x,
                             y = train_y,
                             cv = cv,
                             return_train_score= True   
                              )
    test_mean_score, train_mean_score = pd.DataFrame(scores)[['test_score','train_score']].mean()
    test_scores.append(test_mean_score)
    train_scores.append(train_mean_score)
    


import matplotlib.pyplot as plt

plt.plot(range(len(ns)), train_scores)
plt.plot(range(len(ns)), test_scores)


plt.legend(['train','test'])
plt.show()

pd.DataFrame(
    {'train_scores' : train_scores,
     'test_scores' : test_scores,
     'n_estimators' : ns
    }
)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit, KFold


ns = [100, 200, 300 ]
max_depths = [5, 10, 20, 30 ]
test_scores = []
train_scores = []

for max_depth in max_depths:
    rf = RandomForestRegressor(n_estimators= 200, random_state = 10, max_depth = max_depth)
    cv = KFold(n_splits= 5)

    scores = cross_validate(rf,
                             X = train_x,
                             y = train_y,
                             cv = cv,
                             return_train_score= True   
                              )
    test_mean_score, train_mean_score = pd.DataFrame(scores)[['test_score','train_score']].mean()
    test_scores.append(test_mean_score)
    train_scores.append(train_mean_score)
    


import matplotlib.pyplot as plt

plt.plot(range(len(max_depths)), train_scores)
plt.plot(range(len(max_depths)), test_scores)


plt.legend(['train','test'])
plt.show()

pd.DataFrame(
    {'train_scores' : train_scores,
     'test_scores' : test_scores,
     'max_depth' : max_depths
    }
)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

rf = RandomForestRegressor(n_estimators= 200, random_state = 10, max_depth = 30)
rf.fit(train_x, train_y)
predicted = rf.predict(test_x)


In [ ]:
sample_submission['SalePrice'] = predicted
sample_submission.to_csv('./rf_submission.csv', index = False)

In [ ]:
rf.score(train_x, train_y)